# Postman Collection Converter

This notebook demonstrates how to convert Postman collections to Python API client code using the PostmanCollectionConverter library.

In [1]:
import os
import json
from typing import Dict, List, Optional

from models import PostmanCollection, PostmanRequest
from converter import PostmanCollectionConverter, PostmanRequestConverter
from utils import to_snake_case

## Load a Postman Collection

First, let's load a Postman collection from a JSON file.

In [10]:
# Replace this with your Postman collection path
collection_path = "test.postman_collection.json"

# Define the output directory
output_dir = "./EXPORT/domo_api"

# Load the collection from a file
collection = PostmanCollection.from_file(collection_path)
collection.list_all_headers()

{'host': ['domo-community.domo.com'],
 'accept': ['*/*', 'application/json, text/plain, */*'],
 'sec-ch-ua': ['"Chromium";v="134", "Not:A-Brand";v="24", "Google Chrome";v="134"'],
 'user-agent': ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'],
 'content-type': ['application/json',
  'application/json;charset=UTF-8',
  'application/json;charset=utf-8'],
 'x-csrf-token': ['530f4e52-edef-49f8-9bb6-151535e22469'],
 'sec-fetch-dest': ['empty'],
 'sec-fetch-mode': ['cors'],
 'sec-fetch-site': ['same-origin'],
 'sec-ch-ua-mobile': ['?0'],
 'x-requested-with': ['XMLHttpRequest'],
 'sec-ch-ua-platform': ['"Windows"'],
 'x-domo-requestcontext': ['{"clientToe":"C5MAH5D893-J9NMQ"}',
  '{"clientToe":"C5MAH5D893-55M2H"}',
  '{"clientToe":"C5MAH5D893-INK99"}',
  '{"clientToe":"C5MAH5D893-M0ASA"}',
  '{"clientToe":"C5MAH5D893-5AFLD"}',
  '{"clientToe":"C5MAH5D893-EBRDT"}',
  '{"clientToe":"C5MAH5D893-E9QLV"}',
  '{"clientToe":"C5MAH5D

## Customize the Conversion

Let's define some customization options for specific functions.

In [ ]:
config = {"required_headers" : ['accept' , 'content-type']}
# Customization options for specific endpoints
customizations = {
    # Function name -> customization options
    "get_cards": {
        "default_params": ["urns", "parts", "includeFiltered"],
        "ignored_headers": ["sec-ch-ua", "sec-ch-ua-mobile", "sec-ch-ua-platform"]
    },
    "post_domoweb": {
        "required_headers": ["content-type", "host", "x-csrf-token"],
        "ignored_params": []
    }
}

## Generate Python API Client Files

Now let's generate the Python API client files from the Postman collection.

In [ ]:

# Create the converter
try:
    converter = PostmanCollectionConverter.from_postman_collection(
        postman_path=collection_path,
        export_folder=output_dir,
        customize=customizations
    )
    
    # Generate the files
    converters = converter.generate_conversion_files(
        is_replace=True,             # Replace existing files
        is_include_test_code=True    # Include test functions
    )
    
    print(f"Generated {len(converters)} API client functions.")
except Exception as e:
    print(f"Error generating API client: {str(e)}")

Error generating API client: [Errno 13] Permission denied: 'output/api_client\\post_domoweb.py'


## Test the Generated API Client

Now we can test our generated API client functions.

In [5]:
# Define auth credentials
auth = {
    "base_url": "https://domo-community.domo.com",
    "headers": {
        "x-csrf-token": "your-csrf-token-here",
        "content-type": "application/json;charset=utf-8"
    }
}

# Test the exported functions (commented out to avoid making actual API calls)
# from converter import test_exports
# 
# results = test_exports(
#     export_folder=output_dir,
#     auth=auth,
#     debug_api=True,
#     update_files=True
# )

## Individual Request Conversion

You can also convert individual Postman requests directly.

In [6]:
if 'collection' in locals() and len(collection.requests) > 0:
    # Get a specific request from the collection
    test_request = collection.requests[0]
    
    print(f"Converting request: {test_request.name}")
    
    # Convert with specific configuration
    required_headers = ["content-type", "host"]
    ignored_headers = ["sec-ch-ua", "sec-ch-ua-mobile"]
    default_params = ["urns", "parts"]  # Parameters to expose as function arguments
    ignored_params = ["cacheBuster"]  # Parameters to exclude
    
    # Create a converter for this request
    req_converter = PostmanRequestConverter(Request=test_request)
    
    # Process headers with filtering
    headers = req_converter.generate_headers(required_headers, ignored_headers)
    
    # Process parameters with filtering
    params = req_converter.generate_params(default_params, ignored_params)
    
    print("\nFiltered Headers:")
    for k, v in headers.items():
        print(f"  {k}: {v}")
    
    print("\nFiltered Parameters:")
    for k, v in params.items():
        print(f"  {k}: {v}")
    
    # Export to a file
    if os.path.exists(output_dir):
        filepath = req_converter.export_function(
            export_folder=output_dir,
            prefix="custom_",
            include_test_code=True,
            is_replace=True
        )
        print(f"\nExported to {filepath}")
else:
    print("No collection loaded or collection has no requests.")

Converting request: https://domo-community.domo.com/api/analytics/v1/domoweb

Filtered Headers:
  host: domo-community.domo.com
  accept: application/json, text/plain, */*
  user-agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36
  content-type: application/json;charset=utf-8
  x-csrf-token: 530f4e52-edef-49f8-9bb6-151535e22469
  sec-fetch-dest: empty
  sec-fetch-mode: cors
  sec-fetch-site: same-origin
  x-requested-with: XMLHttpRequest
  sec-ch-ua-platform: "Windows"
  x-domo-requestcontext: {"clientToe":"C5MAH5D893-CKQE4"}

Filtered Parameters:


PermissionError: [Errno 13] Permission denied: 'output/api_client\\custom_post_domoweb.py'